In [ ]:
%%bash
pip install kafka-python

In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json
import re
from datetime import datetime
import nltk
import pickle
from nltk.corpus import stopwords
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import WordNetLemmatizer

In [ ]:
consumer = KafkaConsumer(
    bootstrap_servers='kafka-cluster-kafka-bootstrap.kafka-cluster-project.svc.cluster.local:9092',
    auto_offset_reset='latest',
    value_deserializer=lambda m: json.loads(m.decode('ascii','ignore'))
)

In [ ]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


In [ ]:
producer2 = KafkaProducer(bootstrap_servers='kafka-cluster-kafka-bootstrap.kafka-cluster-project.svc.cluster.local:9092',value_serializer=lambda v: json.dumps(v, cls=NumpyArrayEncoder).encode('utf-8'))

In [ ]:
consumer.subscribe('data_to_predict')

In [ ]:
counter = 1

with open('classifier', 'rb') as model:
    classifier = pickle.load(model)

for message in consumer:
    
    data = np.asarray(message.value["array"])
    result=classifier.predict(data)
    final_comment = {

    "schema": {
        "type": "struct",
        "optional": False,
        "version": 1,
        "fields": [
            {
                "field": "spanid",
                "type": "string",
                "optional": True
            },
            {
                "field": "movieid",
                "type": "string",
                "optional": True
            },
            {
                "field": "userid",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment",
                "type": "string",
                "optional": True
            },
            {
                "field": "status",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment_date",
                "type": "string",
                "optional": True
            }
        ]
    },
    "payload": {
        "spanid": message.value['spanId'],
        "movieid": message.value['movieId'],
        "userid": message.value['userId'],
        "comment": message.value["comment"],
        "comment_date": message.value["comment_date"]   
    }

    }
    
    result = int(result[0])
    
    if result==0:
        final_comment['payload']['status'] = 'normal'
        producer2.send('predicted_normal_1',final_comment)
        print("This looks safe")
    elif result==1:
        final_comment['payload']['status'] = 'anomolous'
        producer2.send('predicted_normal_1',final_comment)
        
        # Send email notification
        comment = final_comment['payload']['comment']
        comment_date = final_comment['payload']['comment_date']
        span_id = final_comment['payload']['spanid']
        movie_id = final_comment['payload']['movieid']
        user_id = final_comment['payload']['userid']
        
        # Please enter corresponding values
        email_address = "WRITE EMAIL HERE"
        api_key = "WRITE API KEY HERE"

        endpoint = "https://api.notification.canada.ca/v2/notifications/email"
        data = {"email_address": email_address,"template_id": "2abfdd23-e228-45a7-8872-8dc6c320faf2", "personalisation": {
          "comment": comment, "comment_date": comment_date, "span_id": span_id, "movie_id": movie_id, "user_id": user_id}}
        headers = {"Authorization": api_key, "Content-Type": "application/json"}
        # print(requests.post(endpoint, data=json.dumps(data), headers=headers).json())
        
        print("This can be Anomolous")
        
    counter = counter + 1
